In [1]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

import random
import nltk
from nltk.stem import WordNetLemmatizer
import json
import pickle
import os

In [2]:
intents_file = open(os.path.join('datasets', 'intents.json')).read()
intents = json.loads(intents_file)

In [3]:
words = []
classes = []
documents = []
ignore_letters = ['!', '?', ',', '.']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)

        documents.append((word, intent['tag']))

        if intent['tag'] not in classes:
            classes.append(intent['tag'])

print(documents)


[(['Hi', 'there'], 'greeting'), (['How', 'are', 'you'], 'greeting'), (['Is', 'anyone', 'there', '?'], 'greeting'), (['Hey'], 'greeting'), (['Hola'], 'greeting'), (['Hello'], 'greeting'), (['Good', 'day'], 'greeting'), (['Bye'], 'goodbye'), (['See', 'you', 'later'], 'goodbye'), (['Goodbye'], 'goodbye'), (['Nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['Till', 'next', 'time'], 'goodbye'), (['Thanks'], 'thanks'), (['Thank', 'you'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['How', 'you', 'could', 'help', 'me', '?'], 'options'), (['What', 'you', 'can', 'do', '?'], 'options'), (['What', 'help', 'you', 'provide', '?'], 'options'), (['How', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['What', 'support', 'is', 'offered'], 'options'), (['How', 'to', 'check', 'Adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['Open', 'adverse', 'drugs', 'module'], 'adverse_drug'), (['Give', 'm

Lemmatize and lowercase each word and remove duplicates

In [4]:
lemmatizer = WordNetLemmatizer()

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]

words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")


47 documents


In [5]:
print(len(classes), 'classes', classes)

print(len(words), "unique lemmatized words", words)
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
87 unique lemmatized words ["'s", 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [6]:
training = []

output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    word_patterns = doc[0]
    word_patterns = [lemmatizer.lemmatize(word.lower()) for word in word_patterns]

    for word in words:
        bag.append(1) if word in word_patterns else bag.append(0)

    output_row = list(output_empty)

    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle the features and make numpy array
random.shuffle(training)
training = np.array(training)

train_x = list(training[:, 0])
train_y = list(training[:, 1])

print('Training data is created')

Training data is created


C:\ProgramData\Anaconda3\envs\deeplearning\lib\site-packages\ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [7]:
train_x = np.array(train_x)
train_y = np.array(train_y)

In [8]:
train_x.shape

(47, 87)

In [9]:
train_y.shape

(47, 9)

In [10]:
model = Sequential()
model.add(Dense(128, input_shape=(train_x.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(train_y.shape[1], activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

hist = model.fit(train_x, train_y, epochs=40, batch_size=8, verbose=1)


Epoch 1/40
6/6 [==============================] - 0s 29ms/step - loss: 2.1992 - accuracy: 0.1064
Epoch 2/40
6/6 [==============================] - 0s 17ms/step - loss: 2.2496 - accuracy: 0.0638
Epoch 3/40
6/6 [==============================] - 0s 51ms/step - loss: 2.1990 - accuracy: 0.1489
Epoch 4/40
6/6 [==============================] - 0s 2ms/step - loss: 2.0848 - accuracy: 0.2128
Epoch 5/40
6/6 [==============================] - 0s 16ms/step - loss: 2.0453 - accuracy: 0.3191
Epoch 6/40
6/6 [==============================] - 0s 31ms/step - loss: 1.9477 - accuracy: 0.3191
Epoch 7/40
6/6 [==============================] - 0s 18ms/step - loss: 1.9268 - accuracy: 0.3404
Epoch 8/40
6/6 [==============================] - 0s 2ms/step - loss: 1.9077 - accuracy: 0.4468
Epoch 9/40
6/6 [==============================] - 0s 29ms/step - loss: 1.8225 - accuracy: 0.5319
Epoch 10/40
6/6 [==============================] - 0s 34ms/step - loss: 1.6668 - accuracy: 0.6170
Epoch 11/40
6/6 [==============

In [11]:
model.save('chatbot_model.h5', hist)